We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [ ]:
import duckdb
from pathlib import Path

candidates = []
search_roots = [
    Path("."),
    Path(".."),
    Path("./data"),
    Path("../data"),
    Path("../../data"),
]

for root in search_roots:
    if root.exists():
        candidates.extend(list(root.glob("nflpa.duckdb")))
        candidates.extend(list(root.glob("**/nflpa.duckdb")))

seen = set()
duckdb_files = []
for f in candidates:
    fp = str(f.resolve())
    if fp not in seen:
        seen.add(fp)
        duckdb_files.append(f.resolve())

print("nflpa.duckdb candidates found", len(duckdb_files))
for i, f in enumerate(duckdb_files[:25]):
    print(i, f)

if not duckdb_files:
    raise RuntimeError("No nflpa.duckdb found near this notebook, run notebook 00 or check where you saved the database")

DB_PATH = duckdb_files[0]
print("using DB_PATH", DB_PATH)

con = duckdb.connect(str(DB_PATH))

con.execute("""
SELECT
  COUNT(*) AS n_tables
FROM (SHOW TABLES)
""").df()